In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Carnarvon,-24.8667,113.6333,84.27,54,0,10.36,AU,2022-02-10 01:31:07
1,1,Yulara,-25.2406,130.9889,87.64,29,64,14.97,AU,2022-02-10 01:34:28
2,2,Albany,42.6001,-73.9662,40.68,66,100,4.56,US,2022-02-10 01:27:25
3,3,Sao Filipe,14.8961,-24.4956,76.96,55,98,5.86,CV,2022-02-10 01:37:45
4,4,Mzimba,-11.9000,33.6000,62.92,96,98,1.54,MW,2022-02-10 01:37:46


In [23]:
city_data_df.dtypes

City_ID         int64
City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date           object
dtype: object

In [24]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [25]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [31]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [32]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [33]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [34]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Carnarvon,-24.8667,113.6333,84.27,54,0,10.36,AU,2022-02-10 01:31:07
1,1,Yulara,-25.2406,130.9889,87.64,29,64,14.97,AU,2022-02-10 01:34:28
3,3,Sao Filipe,14.8961,-24.4956,76.96,55,98,5.86,CV,2022-02-10 01:37:45
6,6,Kaeo,-35.1000,173.7833,77.32,85,100,9.24,NZ,2022-02-10 01:37:47
7,7,Busselton,-33.6500,115.3333,77.04,41,5,6.89,AU,2022-02-10 01:31:03
8,8,Itarema,-2.9248,-39.9167,77.85,77,16,8.28,BR,2022-02-10 01:37:48
9,9,Hithadhoo,-0.6000,73.0833,81.73,74,14,11.86,MV,2022-02-10 01:37:49
11,11,Kapaa,22.0752,-159.3190,79.12,67,75,9.22,US,2022-02-10 01:31:03
12,12,Vaini,-21.2000,-175.2000,86.16,89,40,10.36,TO,2022-02-10 01:32:42
14,14,Orange,33.7000,-117.7675,84.04,22,0,3.44,US,2022-02-10 01:34:59


In [35]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Carnarvon,AU,84.27,-24.8667,113.6333,
1,Yulara,AU,87.64,-25.2406,130.9889,
3,Sao Filipe,CV,76.96,14.8961,-24.4956,
6,Kaeo,NZ,77.32,-35.1000,173.7833,
7,Busselton,AU,77.04,-33.6500,115.3333,
8,Itarema,BR,77.85,-2.9248,-39.9167,
9,Hithadhoo,MV,81.73,-0.6000,73.0833,
11,Kapaa,US,79.12,22.0752,-159.3190,
12,Vaini,TO,86.16,-21.2000,-175.2000,
14,Orange,US,84.04,33.7000,-117.7675,


In [42]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [47]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
# Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
    

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [49]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Carnarvon,AU,84.27,-24.8667,113.6333,Hospitality Carnarvon
1,Yulara,AU,87.64,-25.2406,130.9889,Desert Gardens Hotel - Ayers Rock Resort
3,Sao Filipe,CV,76.96,14.8961,-24.4956,Hotel Xaguate
6,Kaeo,NZ,77.32,-35.1000,173.7833,Bushido Lodge
7,Busselton,AU,77.04,-33.6500,115.3333,Observatory Guest House
8,Itarema,BR,77.85,-2.9248,-39.9167,"Sea Lounge! Bar, Club & Restaurant"
9,Hithadhoo,MV,81.73,-0.6000,73.0833,Scoop Guest House
11,Kapaa,US,79.12,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
12,Vaini,TO,86.16,-21.2000,-175.2000,Keleti Beach Resort
14,Orange,US,84.04,33.7000,-117.7675,Meadows


In [53]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [54]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))